In [ ]:
#integrantes: GABRIEL GOMEZ, EDUARDO DE LA HOZ, STEPHANIA DE LA HOZ

In [ ]:
import numpy as np
import pandas as pd
import scipy.linalg as la
from keras.models import Sequential
from keras.layers.core import Dense

In [ ]:
def cleaner(df,k):
    return df[(abs(df[0]-np.mean(df[0])) <= k*np.std(df[0])) & (abs(df[1]-np.mean(df[1])) <= k*np.std(df[1])) & (abs(df[2]-np.mean(df[2])) <= k*np.std(df[2])) & (abs(df[3]-np.mean(df[3])) <= k*np.std(df[3]))]

In [ ]:
df=pd.read_table('irisdata.txt',skiprows=9,header=None)
dfClean=df.copy()
cat=df.iloc[:,4]
catn=cat.replace([1,2],[0.5,1])
catn=np.array(catn)
dfFlores=df.copy()
df=df.drop(columns=4)
rawdata=np.array(df)
covRawData = np.cov(rawdata.T)
resultRaw = la.eig(covRawData)
eugenVector=resultRaw[1]
eugenValors=resultRaw[0].real

In [ ]:
p_train = 0.80 # Porcentaje de train.

dfFlores['entrenamiento'] = np.random.uniform(0, 1, len(dfFlores)) <= p_train
train, test = dfFlores[dfFlores['entrenamiento']==True], dfFlores[dfFlores['entrenamiento']==False]
dfFlores = dfFlores.drop('entrenamiento', 1)



In [ ]:
modelo=Sequential()
modelo.add(Dense(10,input_dim=2, activation='sigmoid'))
modelo.add(Dense(1, activation='relu'))
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['binary_accuracy'])
